# Helpers

Random assorted helpers

In [ ]:
//| export

import path from "node:path";
import * as ts from "typescript";
import { loadSync } from "@std/dotenv";

In [ ]:
//| export

export const md = async (content: string) => {
  await Deno.jupyter.display({ "text/markdown": content }, { raw: true });
};

In [ ]:
//| export

export const getProjectRoot = (
  dir: string = Deno.cwd(),
  d = 0,
  maxD = 10,
): string => {
  if (d >= maxD) throw new Error("max depth reached");

  try {
    const f = path.join(dir, "deno.json");
    Deno.lstatSync(f);
    return path.dirname(f);
  } catch {
    return getProjectRoot(path.join(dir, "../"), d + 1);
  }
};

In [ ]:
//| export

export const loadEnv = () => {
  loadSync({ envPath: getProjectRoot() + "/.env", export: true });
};

In [ ]:
//| export

// create markdown representation of the directory listing files and subdirectories
export const dirListing = (dir: string, d = 0): string => {
  if (d > 10) {
    return "";
  }

  let md = "";
  for (
    const f of [...Deno.readDirSync(dir)].sort((a, b) =>
      a.name.localeCompare(b.name)
    )
  ) {
    md += `${"  ".repeat(d)}- ${f.name}\n`;
    if (f.isDirectory) {
      md += dirListing(path.join(dir, f.name), d + 1);
    }
  }
  return md;
};

In [ ]:
//| export

import path from "node:path";

export const getNotebooksToProcess = (
  notebookPath: string,
  nbsPath: string,
): string[] => {
  const fullPath = path.isAbsolute(nbsPath)
    ? path.join(nbsPath, notebookPath)
    : path.join(getProjectRoot(), nbsPath, notebookPath);
  const fileInfo = Deno.statSync(fullPath);
  const notebooksToProcess: string[] = [];

  if (fileInfo.isDirectory) {
    // if target is a directory, let's go through all files/directories inside
    for (
      const file of [...Deno.readDirSync(fullPath)].sort((a, b) =>
        a.name.localeCompare(b.name)
      )
    ) {
      if (file.isDirectory) {
        // got another directory? delegate to another getNotebooksToProcess
        const childNbs = getNotebooksToProcess(
          path.join(notebookPath, file.name),
          nbsPath,
        );
        for (const nb of childNbs) {
          notebooksToProcess.push(nb);
        }
        continue;
      }

      // we are only interested in notebooks
      if (!file.name.endsWith(".ipynb")) continue;

      // relative path only, puhleeze
      notebooksToProcess.push(
        path.relative(nbsPath, path.join(fullPath, file.name)),
      );
    }
  } else {
    return [path.relative(nbsPath, fullPath)];
  }

  return notebooksToProcess;
};

In [ ]:
getNotebooksToProcess("nbs/clean.ipynb", ".");

In [ ]:
getNotebooksToProcess(".", ".");

In [ ]:
getNotebooksToProcess(".", path.resolve("."));

In [ ]:
await Deno.jupyter.display({
  "text/markdown": "```md\n" + dirListing("./") + "\n```",
}, { raw: true });

```md
- clean.ipynb
- config.ipynb
- docs.ipynb
- export.ipynb
- jdawg.ipynb
- notebooks.ipynb
- utils.ipynb

```

In [ ]:
//| export

interface SymbolDefinition {
  name: string;
  kind: string;
  signature: string;
  documentation?: string;
  members?: SymbolDefinition[];
}

export function getExportedDefinitions(sourceCode: string): SymbolDefinition[] {
  const sourceFile = ts.createSourceFile(
    "temp.ts",
    sourceCode,
    ts.ScriptTarget.Latest,
    true,
  );

  const definitions: SymbolDefinition[] = [];

  function getNodeSignature(node: ts.Node): string {
    // return sourceCode.slice(node.getStart(), node.getEnd()).trim();
    if (ts.isFunctionDeclaration(node) && node.name) {
      const params = node.parameters
        .map((p) => `${p.name.getText()}: ${p.type?.getText() ?? "any"}`)
        .join(", ");
      const returnType = node.type?.getText() ?? "void";
      return `function ${node.name.text}(${params}): ${returnType}`;
    }

    if (ts.isMethodDeclaration(node) && node.name) {
      const params = node.parameters
        .map((p) => `${p.name.getText()}: ${p.type?.getText() ?? "any"}`)
        .join(", ");
      const returnType = node.type?.getText() ?? "void";
      return `${node.name.getText()}(${params}): ${returnType}`;
    }

    if (ts.isArrowFunction(node)) {
      const params = node.parameters
        .map((p) => `${p.name.getText()}: ${p.type?.getText() ?? "any"}`)
        .join(", ");
      const returnType = node.type?.getText() ?? "any";
      return `(${params}) => ${returnType}`;
    }

    return sourceCode.slice(node.getStart(), node.getEnd()).trim();
  }

  function visit(node: ts.Node) {
    if (
      // @ts-ignore Property 'modifiers' does not exist on type 'Node'.
      node.modifiers?.some(
        (m: ts.Modifier) => m.kind === ts.SyntaxKind.ExportKeyword,
      )
    ) {
      if (ts.isFunctionDeclaration(node) && node.name) {
        definitions.push({
          name: node.name.text,
          kind: "function",
          signature: getNodeSignature(node),
        });
      } else if (ts.isClassDeclaration(node) && node.name) {
        const members = node.members
          .map((member) => {
            if (ts.isMethodDeclaration(member) && member.name) {
              return {
                name: member.name.getText(),
                kind: "method",
                signature: getNodeSignature(member),
              };
            }
            return null;
          })
          .filter(Boolean) as SymbolDefinition[];

        definitions.push({
          name: node.name.text,
          kind: "class",
          signature: getNodeSignature(node),
          members,
        });
      } else if (ts.isInterfaceDeclaration(node) && node.name) {
        definitions.push({
          name: node.name.text,
          kind: "interface",
          signature: getNodeSignature(node),
        });
      } else if (ts.isTypeAliasDeclaration(node) && node.name) {
        definitions.push({
          name: node.name.text,
          kind: "type",
          signature: getNodeSignature(node),
        });
      } else if (ts.isVariableStatement(node)) {
        node.declarationList.declarations.forEach((declaration) => {
          if (
            ts.isVariableDeclaration(declaration) &&
            ts.isIdentifier(declaration.name) &&
            // @ts-ignore Argument of type 'Expression | undefined' is not assignable to parameter of type 'Node'.
            ts.isArrowFunction(declaration.initializer)
          ) {
            // Format: const name = (params) => returnType
            const params = declaration.initializer.parameters
              .map((p) => `${p.name.getText()}: ${p.type?.getText() ?? "any"}`)
              .join(", ");
            const returnType = declaration.initializer.type?.getText() ?? "any";

            definitions.push({
              name: declaration.name.text,
              kind: "function",
              signature:
                `const ${declaration.name.text} = (${params}) => ${returnType}`,
            });
          }
        });
      }
    }

    ts.forEachChild(node, visit);
  }

  visit(sourceFile);
  return definitions;
}

In [ ]:
getExportedDefinitions(`
interface User {
  id: number;
  name: string;
}

export async function j(
  strings: TemplateStringsArray,
  ...expr: string[]
): Promise<void> {
  if (!__jdawg_conversation) {
    throw new Error("JDawg is not initialized.");
  }

  const input = strings.reduce(
    (acc, str, i) => acc + str + (expr[i] || ""),
    "",
  );
  const response = await __jdawg_conversation.complete(input);

  return md(response);
}

export const foo = (p: string): string => {
  return "foo";
};

export type UserRole = 'admin' | 'user';

export function createUser(data: User): User {
  return data;
}

export class UserService {
  async getUser(id: number): Promise<User> {
    return { id, name: 'test' };
  }
  
  async updateUser(user: User): Promise<void> {
    // Update logic
  }
}
`);

In [ ]:
//| export

interface TestInfo {
  name: string;
  body: string;
  location: {
    start: number;
    end: number;
  };
}

export const findDenoTests = (sourceCode: string): TestInfo[] => {
  const sourceFile = ts.createSourceFile(
    "test.ts",
    sourceCode,
    ts.ScriptTarget.Latest,
    true,
  );

  const tests: TestInfo[] = [];

  function visit(node: ts.Node) {
    if (ts.isCallExpression(node)) {
      const expression = node.expression;

      if (
        ts.isPropertyAccessExpression(expression) &&
        ts.isIdentifier(expression.expression) &&
        expression.expression.text === "Deno" &&
        ts.isIdentifier(expression.name) &&
        expression.name.text === "test"
      ) {
        const testName = node.arguments[0];
        const testFn = node.arguments[1];

        if (ts.isStringLiteral(testName)) {
          tests.push({
            name: testName.text,
            body: testFn.getText(),
            location: {
              start: node.getStart(),
              end: node.getEnd(),
            },
          });
        }
      }
    }
    ts.forEachChild(node, visit);
  }

  visit(sourceFile);
  return tests;
};

In [ ]:
import { assert } from "jsr:@std/assert";

Deno.test("findDenoTests", () => {
  assert(
    findDenoTests(`
    Deno.test("test1", () => { assert(1 === 1); });  
  `).length === 1,
  );
  assert(
    findDenoTests(
      `console.log('Deno.test("test1", () => { assert(1 === 1); })')`,
    ).length === 0,
  );
});

In [ ]:
//| export

export const removeDuplicateImports = (sourceCode: string): string => {
  const sourceFile = ts.createSourceFile(
    "temp.ts",
    sourceCode,
    ts.ScriptTarget.Latest,
    true,
  );
  const importNodes: ts.ImportDeclaration[] = [];
  const imports = new Map<string, {
    specifiers: Map<string, string>; // name -> alias
    defaultName?: string;
    namespaceImport?: string;
    typeOnlySpecifiers: Map<string, string>; // name -> alias
    isBareImport?: boolean;
  }>();

  function visit(node: ts.Node) {
    if (ts.isImportDeclaration(node)) {
      importNodes.push(node);
      const source = node.moduleSpecifier.getText().replace(/['"]/g, "");

      if (!imports.has(source)) {
        imports.set(source, {
          specifiers: new Map(),
          typeOnlySpecifiers: new Map(),
          isBareImport: !node.importClause,
        });
      }

      if (!node.importClause) {
        imports.get(source)!.isBareImport = true;
        return;
      }

      const isTypeOnly = node.importClause.isTypeOnly;

      if (
        node.importClause.namedBindings &&
        ts.isNamespaceImport(node.importClause.namedBindings)
      ) {
        imports.get(source)!.namespaceImport =
          node.importClause.namedBindings.name.text;
      } else if (
        node.importClause.namedBindings &&
        ts.isNamedImports(node.importClause.namedBindings)
      ) {
        node.importClause.namedBindings.elements.forEach((element) => {
          const name = element.propertyName?.text || element.name.text;
          const alias = element.propertyName ? element.name.text : undefined;

          if (isTypeOnly || element.isTypeOnly) {
            if (alias) {
              imports.get(source)?.typeOnlySpecifiers.set(name, alias);
            } else {
              imports.get(source)?.typeOnlySpecifiers.set(name, name);
            }
          } else {
            if (alias) {
              imports.get(source)?.specifiers.set(name, alias);
            } else {
              imports.get(source)?.specifiers.set(name, name);
            }
          }
        });
      }
      if (node.importClause.name) {
        imports.get(source)!.defaultName = node.importClause.name.text;
        const target = isTypeOnly
          ? imports.get(source)?.typeOnlySpecifiers
          : imports.get(source)?.specifiers;
        target?.set("default", "default");
      }
    }
    ts.forEachChild(node, visit);
  }

  visit(sourceFile);

  const newImports: string[] = [];
  for (
    const [
      source,
      {
        specifiers,
        typeOnlySpecifiers,
        defaultName,
        namespaceImport,
        isBareImport,
      },
    ] of imports
  ) {
    if (isBareImport) {
      newImports.push(`import '${source}';`);
      continue;
    }

    if (typeOnlySpecifiers.size > 0) {
      let importStr = "import type { ";
      const namedImports = Array.from(typeOnlySpecifiers.entries())
        .sort(([a], [b]) => a.localeCompare(b))
        .map(([name, alias]) => name === alias ? name : `${name} as ${alias}`)
        .join(", ");
      importStr += namedImports;
      importStr += ` } from '${source}';`;
      newImports.push(importStr);
    }

    if (specifiers.size > 0 || namespaceImport) {
      let importStr = "import ";
      if (namespaceImport) {
        importStr += `* as ${namespaceImport}`;
      } else {
        const hasDefault = specifiers.delete("default");
        const namedImports = Array.from(specifiers.entries())
          .sort(([a], [b]) => a.localeCompare(b))
          .map(([name, alias]) => name === alias ? name : `${name} as ${alias}`)
          .join(", ");

        if (hasDefault && defaultName) {
          importStr += `${defaultName} `;
        }
        if (namedImports) {
          importStr += hasDefault
            ? `, { ${namedImports} }`
            : `{ ${namedImports} }`;
        }
      }
      importStr += ` from '${source}';`;
      newImports.push(importStr);
    }
  }

  let result = sourceCode;
  const sortedNodes = importNodes.sort((a, b) => b.getStart() - a.getStart());
  for (const node of sortedNodes) {
    result = result.slice(0, node.getStart()) + result.slice(node.getEnd());
  }

  return newImports.join("\n") + "\n" + result.trim();
};

In [ ]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("removeDuplicateImports", () => {
  assertEquals(
    removeDuplicateImports(`
import { Record as Message } from "tinychat/api/types/chat/tinychat/core/message.ts";        
import type { Config } from "jurassic/config.ts";        
import * as ts from "typescript";        
import { assertEquals } from "jsr:@std/assert";
import path from "node:path";
import "hello"

Deno.test("t1", () => {});

import { assertEquals } from "jsr:@std/assert";
import path from "node:path";

Deno.test("t2", () => {});
`),
    `import { Record as Message } from 'tinychat/api/types/chat/tinychat/core/message.ts';
import type { Config } from 'jurassic/config.ts';
import * as ts from 'typescript';
import { assertEquals } from 'jsr:@std/assert';
import path  from 'node:path';
import 'hello';
Deno.test("t1", () => {});




Deno.test("t2", () => {});`,
  );
});